In [5]:
import numpy as np
from collections import Counter
import math
from numba import jit

In [40]:
(1, 1) * (2, 2)

TypeError: can't multiply sequence by non-int of type 'tuple'

In [39]:
NUM_DICE = 5

In [8]:
def roll_dices(keepers=[]):
    return keepers + [np.random.randint(1, 7) for _ in range(NUM_DICE - len(keepers))]

In [10]:
def select(n, m):
    C = math.factorial(n)/(math.factorial(m)*math.factorial(n-m)) 
    return C

In [11]:
# Calculates the chance of going from one amount of a number to another in one dice roll
def prob1(fro, to):
    n = NUM_DICE - fro 
    m = to - fro
    C = select(n, m)
    prob = C*math.pow(1.0/6.0,(m))*math.pow(5.0/6.0,(n-m)) 
    return prob

In [12]:
# Calculates the chance of going from one amount of a number to another in two dice rolls
def prob2(fro, to):
    prob = 0
    for i in range(fro,to+1):
        prob += prob1(fro,i)*prob1(i,to) 
    return prob

In [13]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [14]:
def cmb_keepers(roll):
    return list(set(list(powerset(roll))))

In [15]:
def upper_bonus(score_card):
    total = 0
    for cat in ['aces', 'twos', 'threes', 'fours', 'fives', 'sixes']:
        total += score_card[cat] if score_card[cat] != -1 else 0
    
    return 35 if total >= 63 else 0

In [16]:
@jit()
def score_3ofkind(roll):
    for e in roll:
        if roll.count(e) >= 3:
            return sum(roll)
    return 0

In [17]:
@jit()
def score_4ofkind(roll):
    for e in roll:
        if roll.count(e) >= 4:
            return sum(roll)
    return 0

In [18]:
@jit()
def score_fullhouse(roll):
    if ((roll.count(list(set(roll))[0]) == 3 and roll.count(list(set(roll))[1]) == 2)
       or (roll.count(list(set(roll))[0]) == 2 and roll.count(list(set(roll))[1]) == 3)):
        return 25
    return 0

In [19]:
def score_smsrtaight(roll):
    roll = list(set(roll))
    roll.sort()
    s = ''.join(str(x) for x in roll)
    for st in ['1234','2345','3456']:
        if st in s:
            return 30
    return 0

In [20]:
def score_lstraight(roll):
    roll = list(set(roll))
    roll.sort()
    s = ''.join(str(x) for x in roll)
    if s == '12345' or s == '23456':
        return 40
    return 0

In [21]:
@jit()
def score_yahtzee(roll):
    return 50 if roll.count(roll[0]) == 5 else 0

In [22]:
# gibt alle einzigartigen Kombinationen von amt verschiedenen Würfeln zurück
def cmb_rolls(amt):
    def _rolls(amt, min_v):
        if amt == 0:
            return [[]]
        
        output = []
        
        for i in range(min_v, 7):
            next_roll = _rolls(amt-1, i)
            for j in range(len(next_roll)):
                next_roll[j] = [i] + next_roll[j]
            output += next_roll
            
        return output
            
    return _rolls(amt, 1)

In [23]:
cmb_rolls(4)

[[1, 1, 1, 1],
 [1, 1, 1, 2],
 [1, 1, 1, 3],
 [1, 1, 1, 4],
 [1, 1, 1, 5],
 [1, 1, 1, 6],
 [1, 1, 2, 2],
 [1, 1, 2, 3],
 [1, 1, 2, 4],
 [1, 1, 2, 5],
 [1, 1, 2, 6],
 [1, 1, 3, 3],
 [1, 1, 3, 4],
 [1, 1, 3, 5],
 [1, 1, 3, 6],
 [1, 1, 4, 4],
 [1, 1, 4, 5],
 [1, 1, 4, 6],
 [1, 1, 5, 5],
 [1, 1, 5, 6],
 [1, 1, 6, 6],
 [1, 2, 2, 2],
 [1, 2, 2, 3],
 [1, 2, 2, 4],
 [1, 2, 2, 5],
 [1, 2, 2, 6],
 [1, 2, 3, 3],
 [1, 2, 3, 4],
 [1, 2, 3, 5],
 [1, 2, 3, 6],
 [1, 2, 4, 4],
 [1, 2, 4, 5],
 [1, 2, 4, 6],
 [1, 2, 5, 5],
 [1, 2, 5, 6],
 [1, 2, 6, 6],
 [1, 3, 3, 3],
 [1, 3, 3, 4],
 [1, 3, 3, 5],
 [1, 3, 3, 6],
 [1, 3, 4, 4],
 [1, 3, 4, 5],
 [1, 3, 4, 6],
 [1, 3, 5, 5],
 [1, 3, 5, 6],
 [1, 3, 6, 6],
 [1, 4, 4, 4],
 [1, 4, 4, 5],
 [1, 4, 4, 6],
 [1, 4, 5, 5],
 [1, 4, 5, 6],
 [1, 4, 6, 6],
 [1, 5, 5, 5],
 [1, 5, 5, 6],
 [1, 5, 6, 6],
 [1, 6, 6, 6],
 [2, 2, 2, 2],
 [2, 2, 2, 3],
 [2, 2, 2, 4],
 [2, 2, 2, 5],
 [2, 2, 2, 6],
 [2, 2, 3, 3],
 [2, 2, 3, 4],
 [2, 2, 3, 5],
 [2, 2, 3, 6],
 [2, 2, 4, 4],
 [2, 2, 4,

In [24]:
calc_cat = {
    'aces' : lambda x : x.count(1),
    'twos' : lambda x : x.count(2)*2,
    'threes' : lambda x : x.count(3)*3,
    'fours' : lambda x : x.count(4)*4,
    'fives' : lambda x : x.count(5)*5,
    'sixes' : lambda x : x.count(6)*6,
    
    '3_kind' : score_3ofkind,
    '4_kind' : score_4ofkind,
    'full_house': score_fullhouse,
    's_straight' : score_smsrtaight,
    'l_straight' : score_lstraight,
    'yahtzee' : score_yahtzee,
    'chance' : lambda x : sum(x)
}

In [25]:
scores = {
    'aces' : -1,
    'twos' : -1,
    'threes' : -1,
    'fours' : -1,
    'fives' : -1,
    'sixes' : -1,
    
    '3_kind' : -1,
    '4_kind' : -1,
    'full_house': -1,
    's_straight' : -1,
    'l_straight' : -1,
    'yahtzee' : 50,
    'chance' : -1,
}

In [26]:
for key in scores.keys():
    if scores[key] == -1:
        print(calc_cat[key]([1,1,1,1,1]))

5
0
0
0
0
0


<ipython-input-16-c7caafff7242>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "score_3ofkind" failed type inference due to: Untyped global name 'sum': cannot determine Numba type of <class 'builtin_function_or_method'>

File "<ipython-input-16-c7caafff7242>", line 5:
def score_3ofkind(roll):
    <source elided>
        if roll.count(e) >= 3:
            return sum(roll)
            ^

  @jit()
c:\python37\lib\site-packages\numba\object_mode_passes.py:178: NumbaWarning: Function "score_3ofkind" was compiled in object mode without forceobj=True.

File "<ipython-input-16-c7caafff7242>", line 2:
@jit()
def score_3ofkind(roll):
^

  state.func_ir.loc))
c:\python37\lib\site-packages\numba\object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest

5
5


c:\python37\lib\site-packages\numba\object_mode_passes.py:178: NumbaWarning: Function "score_4ofkind" was compiled in object mode without forceobj=True.

File "<ipython-input-17-1448eb06207c>", line 2:
@jit()
def score_4ofkind(roll):
^

  state.func_ir.loc))
c:\python37\lib\site-packages\numba\object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit

File "<ipython-input-17-1448eb06207c>", line 2:
@jit()
def score_4ofkind(roll):
^

  state.func_ir.loc))
c:\python37\lib\site-packages\numba\ir_utils.py:2030: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'roll' of function 'score_fullhouse'.

For more information visit

0
0
0
5


In [27]:
def score_card_roll(roll, scorecard):
    scorecard = scorecard.copy()
    
    for key in scores.keys():
        if scorecard[key] == -1:
            scorecard[key] = calc_cat[key](roll)
            
    return scorecard

In [35]:
rolls_scorecards = {}

for roll in cmb_rolls(5):
    roll.sort()
    
    # key = roll as string
    key = ''.join(str(x) for x in roll)
    
    s = score_card_roll(roll, scores)
    
    rolls_scorecards[key] = s

In [36]:
def get_roll_scorecard(roll):
    roll.sort()
    return rolls_scorecards[''.join(str(x) for x in roll)]

In [37]:
def get_final_roll_value(scorecard):
    rolls_scorecards = {}

    for roll in cmb_rolls(5):
        roll.sort()

        # key = roll as string
        key = ''.join(str(x) for x in roll)

        s = score_card_roll(roll, scores)

        rolls_scorecards[key] = s

In [38]:
def max_ev_roll(scorecard, roll):
    card = get_roll_scorecard(roll)
    
    best_key = ''
    best_ev = -1
    
    for key in scorecard.keys():
        if scorecard[key] == -1:
            ev = card[key]
            if ev > best_ev:
                best_key = key
                best_ev = card[key]
            
    return card[key], key

In [32]:
# berechnet die keeper eines gegebenen rolls, welche beim aktuellen schritt bei einem reroll 
# im erwartungswert den höchsten wert haben.
# die funktion müsste sich rerkuriv aufrufen lassen, um so die keeper für 2 rerolls zu berechnen.
# durch rekursive aufrufe mit kopien der scorecard, in welchen die kategorie der jeweiligen
# 'besten' ev rerolls ausgefüllt sind müsste sich die langfristig für den erwartungswert 
# beste strategie berechnen lasseb müssen

def max_ev_keeper(scorecard, roll):
    keepers = cmb_keepers(roll)
    
    cache = {}
    
    max_ev = 0
    best_keepers = roll.copy()
    
    for keeper in keepers:
        keeper = list(keeper)
        
        keeper_cnt = len(keeper)
        if keeper_cnt < 5:
            
            ev = 0
            rerolls = cmb_rolls(5-keeper_cnt)
            for reroll in rerolls:
                reroll += keeper

                key = ''.join(str(x) for x in reroll)

                if key in cache.keys():
                    ev += cache[key]
                else:
                    reroll_ev = max_ev_roll(scorecard, reroll)[0]
                    cache[key] = reroll_ev
                    ev += reroll_ev

            ev /= len(rerolls)
            
        else:
            key = ''.join(str(x) for x in keeper)
            if key in cache.keys():
                ev = cache[key]
            else:
                reroll_ev = max_ev_roll(scorecard, keeper)[0]
                cache[key] = reroll_ev
                ev = reroll_ev
        
        if ev > max_ev:
            max_ev = ev
            best_keepers = keeper
                
                
    return best_keepers, ev

In [33]:
def max_ev_keeper2(scorecard, roll):
    keepers = cmb_keepers(roll)
    
    cache = {}
    
    max_ev = 0
    best_keepers = roll.copy()
    
    
    for keeper in keepers:
        keeper = list(keeper)
        
        keeper_cnt = len(keeper)
        if keeper_cnt < 5:
            ev = 0
            
            rerolls = cmb_rolls(5-keeper_cnt)
            for reroll in rerolls:
                reroll += keeper

                key = ''.join(str(x) for x in reroll)

                if key in cache.keys():
                    ev += cache[key]
                else:
                    reroll_ev = max_ev_keeper(scorecard, reroll)[1]
                    cache[key] = reroll_ev
                    ev += reroll_ev

            ev /= len(rerolls)
            
        else:
            key = ''.join(str(x) for x in keeper)
            if key in cache.keys():
                ev = cache[key]
            else:
                reroll_ev = max_ev_keeper(scorecard, keeper)[1]
                cache[key] = reroll_ev
                ev = reroll_ev
        
        if ev > max_ev:
            max_ev = ev
            best_keepers = keeper

In [34]:
max_ev_keeper2(scores, roll)

In [172]:
scores

{'aces': -1,
 'twos': -1,
 'threes': -1,
 'fours': -1,
 'fives': -1,
 'sixes': -1,
 '3_kind': -1,
 '4_kind': -1,
 'full_house': -1,
 's_straight': -1,
 'l_straight': -1,
 'yahtzee': 50,
 'chance': -1}

In [212]:
roll = roll_dices()

In [213]:
roll

[5, 5, 1, 5, 4]

In [216]:
max_ev_keeper(scores, roll)

20


[5, 5, 5, 4]

In [191]:
max_ev_roll(scores, list((1, 1, 4, 6, 1)))

(13, 'chance')

In [160]:
for i in range(6):
    print(prob1(i, 5))

0.00012860082304526745
0.0007716049382716048
0.0046296296296296285
0.027777777777777776
0.16666666666666666
1.0


In [161]:
for i in range(6):
    print(prob2(i, 5))

0.0026634890885112363
0.008716873380582229
0.02852794924554184
0.0933641975308642
0.3055555555555556
1.0


In [164]:
würfel = 3
(1/6)**würfel

0.0046296296296296285

In [158]:
max_ev_roll(scores, [1,1,1,1,1])

(5, 'chance')

In [141]:
get_roll_scorecard([1,5,1,2,4])

{'aces': 2,
 'twos': 2,
 'threes': 0,
 'fours': 4,
 'fives': 5,
 'sixes': 0,
 '3_kind': 0,
 '4_kind': 0,
 'full_house': 0,
 's_straight': 0,
 'l_straight': 0,
 'yahtzee': 0,
 'chance': 13}

In [147]:
def generate_outcomes(roll):
    outcomes = []
    for cmb in cmb_keepers(roll):
        cmb = list(cmb)
        #print(cmb)
        cmb_rerolls = cmb_rolls(5-len(cmb))

        for reroll in cmb_rerolls:
            outcome = cmb + reroll
            outcomes.append(outcome)
                
    return outcomes

In [148]:
generate_outcomes([1,1,1,3,3])

[[1, 3, 1, 1, 1],
 [1, 3, 1, 1, 2],
 [1, 3, 1, 1, 3],
 [1, 3, 1, 1, 4],
 [1, 3, 1, 1, 5],
 [1, 3, 1, 1, 6],
 [1, 3, 1, 2, 2],
 [1, 3, 1, 2, 3],
 [1, 3, 1, 2, 4],
 [1, 3, 1, 2, 5],
 [1, 3, 1, 2, 6],
 [1, 3, 1, 3, 3],
 [1, 3, 1, 3, 4],
 [1, 3, 1, 3, 5],
 [1, 3, 1, 3, 6],
 [1, 3, 1, 4, 4],
 [1, 3, 1, 4, 5],
 [1, 3, 1, 4, 6],
 [1, 3, 1, 5, 5],
 [1, 3, 1, 5, 6],
 [1, 3, 1, 6, 6],
 [1, 3, 2, 2, 2],
 [1, 3, 2, 2, 3],
 [1, 3, 2, 2, 4],
 [1, 3, 2, 2, 5],
 [1, 3, 2, 2, 6],
 [1, 3, 2, 3, 3],
 [1, 3, 2, 3, 4],
 [1, 3, 2, 3, 5],
 [1, 3, 2, 3, 6],
 [1, 3, 2, 4, 4],
 [1, 3, 2, 4, 5],
 [1, 3, 2, 4, 6],
 [1, 3, 2, 5, 5],
 [1, 3, 2, 5, 6],
 [1, 3, 2, 6, 6],
 [1, 3, 3, 3, 3],
 [1, 3, 3, 3, 4],
 [1, 3, 3, 3, 5],
 [1, 3, 3, 3, 6],
 [1, 3, 3, 4, 4],
 [1, 3, 3, 4, 5],
 [1, 3, 3, 4, 6],
 [1, 3, 3, 5, 5],
 [1, 3, 3, 5, 6],
 [1, 3, 3, 6, 6],
 [1, 3, 4, 4, 4],
 [1, 3, 4, 4, 5],
 [1, 3, 4, 4, 6],
 [1, 3, 4, 5, 5],
 [1, 3, 4, 5, 6],
 [1, 3, 4, 6, 6],
 [1, 3, 5, 5, 5],
 [1, 3, 5, 5, 6],
 [1, 3, 5, 6, 6],
 [1, 3, 6,

In [ ]:
def 

In [ ]:
def max_val_roll(roll):
    

In [ ]:
# calculates best keepers.
# die 'keepers' im 2. reroll basieren auf dem entgültigen roll
# die 'keepers' im 1. reroll basieren auf dem 2. reroll
# die 'keepers' beim roll basieren auf denen beim 1. reroll

def get_best_keepers(score_card, upper_score, roll, rolls_left):
    None